In [1]:
from AnalyzeKeywords import *
import pandas
import pickle
import nltk

defaultdict(<class 'int'>, {2017: 4, 2016: 3, 2018: 10, 2019: 12, 2014: 1})
range(2013, 2019) [0, 1, 0, 3, 4, 10]
defaultdict(<class 'int'>, {2017: 24, 2016: 21, 2014: 6, 2015: 15, 2018: 57, 2019: 60, 2013: 1})
range(2013, 2019) [1, 6, 15, 21, 24, 57]
defaultdict(<class 'int'>, {2015: 4, 2018: 12, 2019: 6, 2016: 2, 2014: 1, 2017: 1})
range(2013, 2019) [0, 1, 4, 2, 1, 12]


<Figure size 640x480 with 1 Axes>

### Scraping

In [2]:
from PyPDF4 import PdfFileReader
from bs4 import BeautifulSoup
import requests
import io
import sys
import csv
from ResearchPaper import ResearchPaper
from Modules import modules
import re

In [3]:
parent_url = 'http://openaccess.thecvf.com/'

In [4]:
conf_url_list = ['http://openaccess.thecvf.com/ICCV2019.py', 
                 'http://openaccess.thecvf.com/CVPR2019.py',
                 'http://openaccess.thecvf.com/CVPR2018.py',
                'http://openaccess.thecvf.com/ICCV2017.py',
                'http://openaccess.thecvf.com/CVPR2017.py',
                'http://openaccess.thecvf.com/CVPR2016.py',
                'http://openaccess.thecvf.com/ICCV2015.py',
                'http://openaccess.thecvf.com/CVPR2015.py',
                'http://openaccess.thecvf.com/CVPR2014.py',
                'http://openaccess.thecvf.com/ICCV2013.py',
                'http://openaccess.thecvf.com/CVPR2013.py',
                'http://openaccess.thecvf.com/ECCV2018.py']
# current_conf_url = 'http://openaccess.thecvf.com/CVPR2019.py'

In [5]:
# Get High Level Scrapping Stats
total_papers_avail = []
num_papers_collected = []
num_err_papers = []
pct_errs = []

In [6]:
#CVPR 2019
# current_conf_url = 'http://openaccess.thecvf.com/CVPR2019.py'
current_conf_url = conf_url_list[0]
current_conf = current_conf_url[-11:-3] # get current conf
research_paper_object_list = []
result = requests.get(current_conf_url)
src = result.content
soup = BeautifulSoup(src, 'lxml')
papers_with_errors = set()

In [7]:
# Get titles
title_list = soup.find_all("dt", class_="ptitle")
titles = [title.get_text() for title in title_list]
total_papers_avail.append(len(titles))

In [10]:
# Set Unique IDs
# If CVPR, then * 2877
# If ICCV, then * 4228
# If ECCV, then * 3228
if current_conf[0:4] == 'CVPR':
    unique_ids = [2019 * 2877 * i for i in range(1,len(title_list)+1)]
elif current_conf[0:4] == 'ICCV':
    unique_ids = [2019 * 4228 * i for i in range(1,len(title_list)+1)]
else:
    unique_ids = [2019 * 3228 * i for i in range(1,len(title_list)+1)]

In [11]:
# Remove all line breaks
for e in soup.findAll('br'):
    e.extract()

In [12]:
raw_papers = soup.find_all(class_='bibref')
authors = []
year = []
for raw_paper in raw_papers:
    single_paper = raw_paper.text.split('\n')
    for text in single_paper:
        if text != '' and len(text.split('=')) > 1:
            header = text.split('=')[0]
            body = text.split('=')[1]
            if header == 'author ':
                # Clean out braces and split according to 'and'
                cleaned_body = body.replace(' {','').replace('}','').split(' and ')
                paper_authors = []
                for name in cleaned_body:
                    last_name = name.split(',')[0]
                    first_name = name.split(',')[1]
#                     print(first_name + ' ' + last_name)
                    paper_authors.append(first_name + ' ' + last_name)
                authors.append(paper_authors)
    # Get title if the above method doesn't work
    #         if header = 'title':
    #             # Clean out braces, commas and spaces
    #             cleaned_body = body.replace('{','').replace('}','').replace(',','')[1:]
    #             titles.append(cleaned_body)
            if header == 'year ':
                year.append(body.replace(' {','').replace('}',''))

In [13]:
# Get paper texts and PDF Urls

pdf_list = soup.find_all(href=re.compile("/papers/"))
paper_text = []
pdf_urls = []
#Check length of PDF List matches length of UNIQUE IDS
if len(pdf_list) == len(unique_ids):
    for count, pdf in enumerate(pdf_list):
        try:
            pdf_url = parent_url + pdf_list[count].get('href')
            paper_text.append(modules.pdf_string_from_url(pdf_url))
            pdf_urls.append(pdf_url)
            print('**** PAPER NUMBER [{}] WITH URL [{}] ADDED ****'.format(unique_ids[count],pdf_url))
        except:
            paper_text.append('FAILED')
            pdf_urls.append('FAILED')
            papers_with_errors.add(unique_ids[count])
            print('**** PAPER NUMBER [{}] WITH URL [{}] FAILED ****'.format(unique_ids[count],pdf_url))
else:
    print('**** LENGTH OF PDF LIST [{}] DOES NOT MATCH LENGTH OF UNIQUE IDS [{}] ****'.format(len(pdf_list),len(unique_ids)))

**** PAPER NUMBER [8536332] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Rossler_FaceForensics_Learning_to_Detect_Manipulated_Facial_Images_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [17072664] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Lu_DeepVCP_An_End-to-End_Deep_Neural_Network_for_Point_Cloud_Registration_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [25608996] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Gadelha_Shape_Reconstruction_Using_Differentiable_Projections_and_Deep_Priors_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [34145328] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Larsson_Fine-Grained_Segmentation_Networks_Self-Supervised_Segmentation_for_Improved_Long-Term_Visual_Localization_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [42681660] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Yang_SANet_Scene_Agnostic_Network_for_Camera_Localization_ICCV_2019_paper.

**** PAPER NUMBER [367062276] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Zhang_Towards_Adversarially_Robust_Object_Detection_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [375598608] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Zhao_Automatic_and_Robust_Skull_Registration_Based_on_Discrete_Uniformization_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [384134940] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Peng_Few-Shot_Image_Recognition_With_Knowledge_Transfer_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [392671272] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Wray_Fine-Grained_Action_Retrieval_Through_Multiple_Parts-of-Speech_Embeddings_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [401207604] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Wang_Vehicle_Re-Identification_in_Aerial_Imagery_Dataset_and_Approach_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [409743936] WITH 

**** PAPER NUMBER [725588220] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Zhou_Relation_Parsing_Neural_Network_for_Human-Object_Interaction_Detection_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [734124552] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Girdhar_DistInit_Learning_Video_Representations_Without_a_Single_Labeled_Video_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [742660884] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Sener_Zero-Shot_Anticipation_for_Instructional_Activities_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [751197216] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Li_Making_the_Invisible_Visible_Action_Recognition_Through_Walls_and_Occlusions_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [759733548] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Xu_Recursive_Visual_Sound_Separation_Using_Minus-Plus_Net_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [

**** PAPER NUMBER [1092650496] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Nguyen_Anomaly_Detection_in_Video_Sequence_With_Appearance-Motion_Correspondence_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [1101186828] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Xie_Exploring_Randomly_Wired_Neural_Networks_for_Image_Recognition_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [1109723160] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Chen_Progressive_Differentiable_Architecture_Search_Bridging_the_Depth_Gap_Between_Search_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [1118259492] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Zheng_Multinomial_Distribution_Learning_for_Effective_Neural_Architecture_Search_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [1126795824] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Howard_Searching_for_MobileNetV3_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NU

**** PAPER NUMBER [1459712772] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Gong_Memorizing_Normality_to_Detect_Anomaly_Memory-Augmented_Deep_Autoencoder_for_Unsupervised_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [1468249104] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Li_Topological_Map_Extraction_From_Overhead_Images_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [1476785436] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Zhang_Exploiting_Temporal_Consistency_for_Real-Time_Video_Depth_Estimation_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [1485321768] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Zhao_The_Sound_of_Motions_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [1493858100] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Jo_SC-FEGAN_Face_Editing_Generative_Adversarial_Network_With_Users_Sketch_and_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [1502394432] WITH URL 

...

**** PAPER NUMBER [7571726484] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Xie_Image_Inpainting_With_Learnable_Bidirectional_Attention_Maps_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [7580262816] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Ehret_Joint_Demosaicking_and_Denoising_by_Fine-Tuning_of_Bursts_of_Raw_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [7588799148] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Kupyn_DeblurGAN-v2_Deblurring_Orders-of-Magnitude_Faster_and_Better_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [7597335480] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Ke_Reflective_Decoding_Network_for_Image_Captioning_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [7605871812] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Vered_Joint_Optimization_for_Cooperative_Image_Captioning_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [7614408144] WITH URL [http://ope

**** PAPER NUMBER [7938788760] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Brazil_M3D-RPN_Monocular_3D_Region_Proposal_Network_for_Object_Detection_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [7947325092] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Behley_SemanticKITTI_A_Dataset_for_Semantic_Scene_Understanding_of_LiDAR_Sequences_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [7955861424] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Yogamani_WoodScape_A_Multi-Task_Multi-Camera_Fisheye_Dataset_for_Autonomous_Driving_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [7964397756] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Doan_Scalable_Place_Recognition_Under_Appearance_Change_for_Autonomous_Driving_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [7972934088] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Codevilla_Exploring_the_Limitations_of_Behavior_Cloning_for_Autonomous_Driving

**** PAPER NUMBER [8297314704] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Cao_Hierarchical_Shot_Detector_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [8305851036] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Li_Few-Shot_Learning_With_Global_Class_Representations_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [8314387368] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Noh_Better_to_Follow_Follow_to_Be_Better_Towards_Precise_Supervision_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [8322923700] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Li_Weakly_Supervised_Object_Detection_With_Segmentation_Collaboration_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [8331460032] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Najibi_AutoFocus_Efficient_Multi-Scale_Inference_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [8339996364] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/pap

**** PAPER NUMBER [8664376980] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Moon_Camera_Distance-Aware_Top-Down_Approach_for_3D_Multi-Person_Pose_Estimation_From_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [8672913312] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Lee_Context-Aware_Emotion_Recognition_Networks_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [8681449644] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Qian_Aggregation_via_Separation_Boosting_Facial_Landmark_Detector_With_Semi-Supervised_Style_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [8689985976] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Kuhnke_Deep_Head_Pose_Estimation_Using_Synthetic_Images_and_Partial_Adversarial_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [8698522308] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Sassoon_Flare_in_Interference-Based_Hyperspectral_Cameras_ICCV_2019_paper.pdf] ADDED ****
*

**** PAPER NUMBER [9022902924] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Liu_Few-Shot_Unsupervised_Image-to-Image_Translation_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [9031439256] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Yang_Very_Long_Natural_Scenery_Image_Prediction_by_Outpainting_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [9039975588] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Mehta_Scaling_Recurrent_Models_via_Orthogonal_Approximations_in_Tensor_Trains_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [9048511920] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Kim_A_Deep_Cybersickness_Predictor_Based_on_Brain_Signal_Analysis_for_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [9057048252] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/papers/Wu_Learning_With_Unsure_Data_for_Medical_Image_Diagnosis_ICCV_2019_paper.pdf] ADDED ****
**** PAPER NUMBER [9065584584] WITH URL [htt

In [14]:
# Grab all abstracts

abstracts = []
abs_list = soup.find_all(class_='ptitle')

# Check that length of the abstract list matches that of the UNIQUE IDS
if len(abs_list) == len(unique_ids):
    for count, abstract in enumerate(abs_list):
        try:
            abstract_url = parent_url + abs_list[count].contents[0].get('href')
            abs_src = requests.get(abstract_url).content
            abs_soup = BeautifulSoup(abs_src, 'lxml')
            abstracts.append(abs_soup.find_all(id='abstract')[0].text)
        except:
            abstracts.append('FAILED')
            papers_with_errors.add(unique_ids[count])
            print('**** PAPER NUMBER [{}] WITH URL [{}] FAILED ****'.format(unique_ids[count],abstract_url))
else:
        print('**** LENGTH OF ABS LIST [{}] DOES NOT MATCH LENGTH OF UNIQUE IDS [{}] ****'.format(len(abs_list),len(unique_ids)))

**** PAPER NUMBER [102435984] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/html/Ye_Adversarial_Robustness_vs._Model_Compression_or_Both_ICCV_2019_paper.html] FAILED ****
**** PAPER NUMBER [657297564] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/html/Lee_Talking_With_Hands_16.2M_A_Large-Scale_Dataset_of_Synchronized_Body-Finger_ICCV_2019_paper.html] FAILED ****
**** PAPER NUMBER [4993754220] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/html/Barroso-Laguna_Key.Net_Keypoint_Detection_by_Handcrafted_and_Learned_CNN_Filters_ICCV_2019_paper.html] FAILED ****
**** PAPER NUMBER [7272954864] WITH URL [http://openaccess.thecvf.com/content_ICCV_2019/html/Wang_Miss_Detection_vs._False_Alarm_Adversarial_Learning_for_Small_Object_ICCV_2019_paper.html] FAILED ****


In [15]:
for idx, unique_id in enumerate(unique_ids):
    if unique_id not in papers_with_errors:
        this_paper = ResearchPaper(unique_ids[idx], year[idx], titles[idx], pdf_urls[idx], authors[idx], paper_text[idx], abstracts[idx])
        research_paper_object_list.append(this_paper)

In [16]:
num_papers_collected.append(len(research_paper_object_list))
num_err_papers.append(len(papers_with_errors))
pct_errs.append(num_err_papers[0]/total_papers_avail[0]*100)

In [17]:
# Write to CSV
with open(current_conf + '.csv', 'w') as new_csv:
    writer = csv.writer(new_csv)
    header = ['unique_id', 'year', 'title', 'authors', 'pdf_url', 'paper_text', 'abstract']
    writer.writerow(header)

    for paper in research_paper_object_list:
        row = [paper.unique_id, paper.year, paper.title, paper.authors, paper.pdf_url, paper.paper_text, paper.abstract]
        writer.writerow(row)

In [18]:
# # get all the papers
# for idx,raw_paper in enumerate(raw_papers_list):
#     title = raw_paper.find(class_="title").text
#     author_string = raw_paper.find(class_="authors").contents[0]

#     authors = []
#     #Get rid of leading new lines and spaces in authors string
#     split_list = author_string.split('\n')
#     for string in split_list:
#         string = string.strip()
#         if string != "": authors.append(string)


#     #Manage edge case of different PDF href text in 2015
#     if year == 2015:
#         pdf_url = raw_paper.find('a', href=True, text='pdf')['href']
#     else:
#         pdf_url = raw_paper.find('a', href=True, text='Download PDF')['href']

#     abstract_url = raw_paper.find('a', href=True, text='abs')['href']
#     abs_src = requests.get(abstract_url).content
#     soup = BeautifulSoup(abs_src, 'lxml')
#     abstract = soup.find("div", class_="abstract").text.strip()
#     abstract = abstract.replace('\n', '')

#     #Pull unique ID
#     unique_id = pdf_url.split("/")[4]

#     #Pull PDF text
#     #TODO make sure new method approach is working
#     try: 
#         paper_text = modules.pdf_string_from_url(pdf_url)

#         this_paper = ResearchPaper(unique_id, year, title, pdf_url, authors, paper_text, abstract)
#         research_paper_object_list.append(this_paper)
#         #some feedback that stuff is happening
#         print('{} / {} {}'.format(idx+1, len(raw_papers_list), this_paper.title))
#     except KeyboardInterrupt:
#         quit()
#     except:
#         print('**** PAPER WITH TITLE [{}] FAILED ****'.format(title))

# with open('icml' + str(year) + '.csv', 'w') as new_csv:
#     writer = csv.writer(new_csv)
#     header = ['unique_id', 'year', 'title', 'authors', 'pdf_url', 'paper_text', 'abstract']
#     writer.writerow(header)

#     for paper in research_paper_object_list:
#         row = [paper.unique_id, paper.year, paper.title, paper.authors, paper.pdf_url, paper.paper_text, paper.abstract]
#         writer.writerow(row)

### Analysis

In [19]:
# papers = pandas.read_pickle('papers.p')

In [20]:
# papers[0].authors